In [ ]:
import os
WEBHOOK_URL = os.environ['SLACK_WEBHOOK_URL']
GOOD_CHANNEL_ID = os.environ['SLACK_GOOD_CHANNEL_ID']
DANGER_CHANNEL_ID = os.environ['SLACK_DANGER_CHANNEL_ID']

In [ ]:
def send_slack(message, text, color='good'):
    if color == 'good':
        channel_id = GOOD_CHANNEL_ID
        message = f':grinning: {message}'
    if color == 'danger':
        channel_id = DANGER_CHANNEL_ID
        message = f':slightly_frowning_face: {message}'

    import requests
    requests.post(
        WEBHOOK_URL,
        json={
            'channel': channel_id,
            'username': 'Trading Notifications',
            'icon_url': 'https://ethereum.org/favicon-32x32.png',
            'text': message,
            'mrkdwn': True,
            'attachments': [{
                'title': '→ Trading Operations',
                'title_link': 'https://jhub.name/user/sheng_wei/lab',
                'text': text,
                'color': color,
            }]
        }
    )

# A Trader Stabilize Fiat Value of Crypto Currency

In [ ]:
TEST_RATIO = 1

In [ ]:
SAMPLE_INTERVAL = 60
HAPPY_COOLING_TIME = 60
SAD_COOLING_TIME = 60 * 6

NOTIFY_RATE = 60 * 8
TRADE_RATE = 10 << 10
BACKUP_RATE = 1

TRADE_UNIT = 0.001
INIT_BUY_JPY = 1000
INIT_SELL_JPY = 1000
MAX_TRADED_JPY = 800000

TRADABLE_UNIT_CC_GAIN_JPY = 1000 # TODO: self-adaptive by trend analysis
TRADABLE_UNIT_CC_LOSS_JPY = 3000

MIN_UNIT_CC_INIT_TRADE_JPY = 26500
MAX_UNIT_CC_INIT_TRADE_JPY = MIN_UNIT_CC_INIT_TRADE_JPY << 10
MAX_UNIT_CC_LOSS_JPY = MAX_UNIT_CC_INIT_TRADE_JPY

MAX_GAIN_JPY = MAX_TRADED_JPY << 10 # useless
MAX_LOSS_JPY = MAX_TRADED_JPY << 10 # useless
MAX_TOTAL_LOST_JPY = MAX_TRADED_JPY << 10

In [ ]:
from datetime import datetime
import json, sys, time
import python_bitbankcc

In [ ]:
pub = python_bitbankcc.public()

## Simulated One

In [ ]:
from unittest.mock import Mock
prv = Mock()

## Actual One

In [ ]:
# import os
# prv = python_bitbankcc.private(
#     os.environ['BITBANK_API_KEY'],
#     os.environ['BITBANK_API_SECRET']
# )

## Trading Operations

In [ ]:
def sell_all():
    now_ticker = pub.get_ticker('eth_jpy')
    now_sell_jpy = int(now_ticker.get('buy', '0'))
    now_buy_jpy = int(now_ticker.get('sell', '1'))

    please_sell_amount = TRADE_UNIT * bought_unit_amount
    prv.order(
        pair='eth_jpy', price=str(now_sell_jpy), amount=str(please_sell_amount), side='sell', order_type='market'
    )

    bought_amount = TRADE_UNIT * bought_unit_amount
    total_gain_jpy = total_gained_jpy + now_sell_jpy * bought_amount - used_jpy
    with open('sample-history.txt', 'a', 1) as fsh:
        fsh.write(
            f'BUY ETH for {now_buy_jpy} JPY: SELL ETH for {now_sell_jpy} JPY\n'
            f'GAIN {total_gain_jpy} JPY in total at {datetime.now()}\n\n'
        )
    print(
        f'BUY ETH for {now_buy_jpy} JPY: SELL ETH for {now_sell_jpy} JPY\n'
        f'GAIN {total_gain_jpy} JPY in total at {datetime.now()}\n\n',
        end=''
    )

    used_jpy_after = used_jpy * (bought_unit_amount - please_sell_unit_amount) / bought_unit_amount

    avg_jpy = used_jpy / bought_amount
    diff_jpy = now_sell_jpy - avg_jpy
    gained_jpy = diff_jpy * please_sell_amount
    with open('transaction-history.txt', 'a', 1) as fth:
        fth.write(
            f'SOLD {please_sell_amount:.4f}=+{bought_amount:.4f}({used_jpy/MAX_TRADED_JPY*100:.2f}%)'
            f'-{bought_amount-please_sell_amount:.4f}(0.00%) ETH'
            f' for {diff_jpy:.2f}={now_sell_jpy}-{avg_jpy:.2f} JPY：BUY ETH for {now_buy_jpy} JPY\n'
            f'GAINED {gained_jpy} JPY: GAINED {total_gained_jpy + gained_jpy} JPY in total at {datetime.now()}\n\n'
        )
    print(
        f'SOLD {please_sell_amount:.4f}=+{bought_amount:.4f}({used_jpy/MAX_TRADED_JPY*100:.2f}%)'
        f'-{bought_amount-please_sell_amount:.4f}(0.00%) ETH'
        f' for {diff_jpy:.2f}={now_sell_jpy}-{avg_jpy:.2f} JPY：BUY ETH for {now_buy_jpy} JPY\n'
        f'GAINED {gained_jpy} JPY: GAINED {total_gained_jpy + gained_jpy} JPY in total at {datetime.now()}\n\n',
        end=''
    )

    return {
        'sample_number': sample_number,
        'buy_count': buy_count,
        'sell_count': sell_count,
        'bought_unit_amount': bought_unit_amount,
        'used_jpy': used_jpy,
        'please_buy_unit_amount': please_buy_unit_amount,
        'please_sell_unit_amount': please_sell_unit_amount,
        'cooling_time': cooling_time,
        'total_gained_jpy': total_gained_jpy
    }

## Start Trading

In [ ]:
ROBOT_NAME = 'Short-Term Robot'
send_slack(
    f'{"Mock of " if isinstance(prv, Mock) else ""}{ROBOT_NAME} started', 'Power by https://jhub.name/', 'good'
)

In [ ]:
import os
import yaml

PATH = os.getenv('ROBOT_PATH') or '.'
STATUS_FILE = f'{PATH}/robot-status.yaml'
TMPFILE = f'{PATH}/_tmp'


def read_status():
    if os.path.exists(STATUS_FILE):
        with open(STATUS_FILE, 'r') as f:
            return yaml.load(f, Loader=yaml.FullLoader)
    else:
        return {}


def write_status(status):
    with open(TMPFILE, 'w') as f:
        yaml.dump(status, f)
    os.replace(TMPFILE, STATUS_FILE)

In [ ]:
status = read_status()
sample_number = status.get('sample_number', 0)
trade_count = status.get('trade_count', 0)
buy_count = status.get('buy_count', 0)
sell_count = status.get('sell_count', 0)
bought_unit_amount = status.get('bought_unit_amount', 0)
used_jpy = status.get('used_jpy', 0)
bought_average_jpy = status.get('bought_average_jpy', MAX_UNIT_CC_INIT_TRADE_JPY)
sold_unit_amount = status.get('sold_unit_amount', 0)
got_jpy = status.get('got_jpy', 0)
sold_average_jpy = status.get('sold_average_jpy', MAX_UNIT_CC_INIT_TRADE_JPY)
please_buy_unit_amount = status.get('please_buy_unit_amount', 0)
please_sell_unit_amount = status.get('please_sell_unit_amount', 0)
cooling_time = status.get('cooling_time', HAPPY_COOLING_TIME)
total_gained_jpy = status.get('total_gained_jpy', 0)

In [ ]:
import math
from scipy import optimize as opt
from numpy.lib import scimath

def get_trade_unit_amount(now_trade_jpy, traded_unit_amount, traded_jpy, tradable_unit_cc_diff_jpy, init_jpy):
    if traded_unit_amount == 0:
        return 1
    avg_jpy = traded_jpy / (TRADE_UNIT * traded_unit_amount)
    diff_jpy = abs(now_trade_jpy - avg_jpy)
    min_diff_jpy = tradable_unit_cc_diff_jpy / scimath.log(MAX_TRADED_JPY / init_jpy)
    return math.ceil(
        traded_jpy * opt.fsolve(
            lambda c: [
                (
                    diff_jpy - min_diff_jpy * scimath.log(1 + c[0]) / (avg_jpy * c[0]) * (avg_jpy * c[0] + now_trade_jpy)
                ).real
            ],
            [-0.5]
        )[0] / now_trade_jpy / TRADE_UNIT
    )

In [ ]:
from decimal import Decimal
with open('sample-history.txt', 'a', 1) as fsh, open('transaction-history.txt', 'a', 1) as fth:
    while True:
        bought_unit_amounts = [
            int(Decimal(asset.get('free_amount', '0')) / Decimal(repr(TRADE_UNIT))) for asset in prv.get_asset().get('assets', []) \
                if asset.get('asset', '') == 'eth'
        ] if not isinstance(prv, Mock) else []

        if len(bought_unit_amounts) > 0:
            bought_unit_amount_before = bought_unit_amount

            bought_unit_amount = bought_unit_amounts[0]
            used_jpy *= bought_unit_amount / bought_unit_amount_before

        now_ticker = pub.get_ticker('eth_jpy')
        now_sell_jpy = int(now_ticker.get('buy', '0'))
        now_buy_jpy = int(now_ticker.get('sell', '1'))

        bought_amount = TRADE_UNIT * bought_unit_amount

        if sample_number % TRADE_RATE == 0 and bought_unit_amount > 0 and now_sell_jpy > used_jpy/bought_amount:
            please_sell_unit_amount = 1
            cooling_time = HAPPY_COOLING_TIME

        if sold_unit_amount == 0 and bought_unit_amount > 0 and now_sell_jpy > used_jpy/bought_amount:
            sample_number = 0
            please_sell_unit_amount = min(math.ceil(INIT_SELL_JPY / now_sell_jpy / TRADE_UNIT), bought_unit_amount)
            cooling_time = HAPPY_COOLING_TIME

        if sold_unit_amount > 0 and now_sell_jpy > got_jpy / (TRADE_UNIT * sold_unit_amount):
            trade_unit_amount = min(
                get_trade_unit_amount(now_sell_jpy, sold_unit_amount, got_jpy, TRADABLE_UNIT_CC_GAIN_JPY, INIT_SELL_JPY), bought_unit_amount
            )
            if trade_unit_amount > 0:
                sample_number = 0
                please_sell_unit_amount = trade_unit_amount
                cooling_time = HAPPY_COOLING_TIME

        if now_sell_jpy * bought_amount - used_jpy > MAX_GAIN_JPY:
            sample_number = 0
            please_sell_unit_amount = bought_unit_amount
            cooling_time = HAPPY_COOLING_TIME

        if now_sell_jpy * bought_amount - used_jpy < -MAX_LOSS_JPY:
            sample_number = 0
            please_sell_unit_amount = bought_unit_amount
            cooling_time = SAD_COOLING_TIME

        if bought_unit_amount > 0 and now_sell_jpy < used_jpy/bought_amount - MAX_UNIT_CC_LOSS_JPY and used_jpy/MAX_TRADED_JPY > 90/100:
            sample_number = 0
            please_sell_unit_amount = bought_unit_amount
            cooling_time = SAD_COOLING_TIME

        if bought_unit_amount > 0 and now_sell_jpy < MIN_UNIT_CC_INIT_TRADE_JPY:
            please_sell_unit_amount = bought_unit_amount
            cooling_time = SAD_COOLING_TIME

        if please_sell_unit_amount > 0:
            please_sell_amount = TRADE_UNIT * please_sell_unit_amount
            prv.order(
                pair='eth_jpy', price=str(now_sell_jpy), amount=str(please_sell_amount), side='sell', order_type='market'
            )

            bought_amount = TRADE_UNIT * bought_unit_amount
            gain_jpy = now_sell_jpy * bought_amount - used_jpy
            total_gain_jpy = total_gained_jpy + gain_jpy
            fsh.write(
                f'BUY ETH for {now_buy_jpy} JPY: SELL ETH for {now_sell_jpy} JPY\n'
                f'GAIN {total_gain_jpy:+.2f}={total_gained_jpy:+.2f}{gain_jpy:+.2f} JPY in total at {datetime.now()}\n\n'
            )

            used_jpy_after = used_jpy * (bought_unit_amount - please_sell_unit_amount) / bought_unit_amount

            avg_jpy = used_jpy / bought_amount
            diff_jpy = now_sell_jpy - avg_jpy
            gained_jpy = diff_jpy * please_sell_amount
            total_gained_jpy += gained_jpy
            fth.write(
                f'SOLD {please_sell_amount:.4f}=+{bought_amount:.4f}({used_jpy/MAX_TRADED_JPY*100:.2f}%)'
                f'-{bought_amount-please_sell_amount:.4f}({used_jpy_after/MAX_TRADED_JPY*100:.2f}%) ETH for {diff_jpy:.2f}'
                f'={now_sell_jpy}-{avg_jpy:.2f} JPY：BUY ETH for {now_buy_jpy} JPY: NO.{trade_count+1}-{sell_count+1}\n'
                f'GAINED {gained_jpy:+.2f} JPY: GAINED {total_gained_jpy:+.2f} JPY in total at {datetime.now()}\n\n'
            )

            send_slack(
                f'Gained {gained_jpy:+.2f} JPY. Gained {total_gained_jpy:+.2f} JPY in total'
                f' by {"Mock of " if isinstance(prv, Mock) else ""}{ROBOT_NAME}.',
                'Power by https://jhub.name/', 'good' if total_gained_jpy > 0 else 'danger'
            )

            bought_unit_amount_before = bought_unit_amount

            sell_count += 1
            bought_unit_amount -= please_sell_unit_amount
            used_jpy *= bought_unit_amount / bought_unit_amount_before
            sold_unit_amount += please_sell_unit_amount
            got_jpy += now_sell_jpy * please_sell_amount
            sold_average_jpy = got_jpy / (TRADE_UNIT * sold_unit_amount)
            please_buy_unit_amount = 0
            please_sell_unit_amount = 0
            cooling_time = HAPPY_COOLING_TIME

            if bought_unit_amount == 0:
                trade_count += 1
                buy_count = sell_count = 0

            if bought_unit_amount == 0 and total_gained_jpy > MAX_GAIN_JPY << 2 and not isinstance(prv, Mock):
                from unittest.mock import Mock
                prv = Mock()
                send_slack(
                    f'{ROBOT_NAME} is turned to Mock', 'Power by https://jhub.name/', 'good'
                )

            write_status({
                'sample_number': sample_number + 1,
                'trade_count': trade_count,
                'buy_count': buy_count,
                'sell_count': sell_count,
                'bought_unit_amount': bought_unit_amount,
                'used_jpy': used_jpy,
                'bought_average_jpy': bought_average_jpy,
                'sold_unit_amount': sold_unit_amount,
                'got_jpy': got_jpy,
                'sold_average_jpy': sold_average_jpy,
                'please_buy_unit_amount': please_buy_unit_amount,
                'please_sell_unit_amount': please_sell_unit_amount,
                'cooling_time': cooling_time,
                'total_gained_jpy': total_gained_jpy,
                'trade_unit': TRADE_UNIT
            })

            if total_gained_jpy < -MAX_TOTAL_LOST_JPY:
                break

            sample_number += 1
            time.sleep(cooling_time/TEST_RATIO)
            continue

        init_buy_unit_amount = math.ceil(INIT_BUY_JPY / now_buy_jpy / TRADE_UNIT)

        if bought_unit_amount < init_buy_unit_amount and now_buy_jpy > MIN_UNIT_CC_INIT_TRADE_JPY and now_buy_jpy < MAX_UNIT_CC_INIT_TRADE_JPY \
                and used_jpy + now_buy_jpy * TRADE_UNIT <= MAX_TRADED_JPY:
            sample_number = 0
            please_buy_unit_amount = init_buy_unit_amount - bought_unit_amount

        if bought_unit_amount >= init_buy_unit_amount and now_buy_jpy < used_jpy/bought_amount:
            trade_unit_amount = get_trade_unit_amount(now_buy_jpy, bought_unit_amount, used_jpy, TRADABLE_UNIT_CC_LOSS_JPY, INIT_BUY_JPY)
            while trade_unit_amount > 0 and used_jpy + now_buy_jpy * TRADE_UNIT * trade_unit_amount > MAX_TRADED_JPY:
                trade_unit_amount >>= 1
            if trade_unit_amount > 0:
                sample_number = 0
                please_buy_unit_amount = trade_unit_amount

        if please_buy_unit_amount > 0:
            please_buy_amount = TRADE_UNIT * please_buy_unit_amount
            prv.order(
                pair='eth_jpy', price=str(now_buy_jpy), amount=str(please_buy_amount), side='buy', order_type='market'
            )

            used_jpy_before = used_jpy

            buy_count += 1
            bought_unit_amount += please_buy_unit_amount
            used_jpy += now_buy_jpy * please_buy_amount
            bought_average_jpy = used_jpy / (TRADE_UNIT * bought_unit_amount)
            sold_unit_amount = 0
            got_jpy = 0
            please_buy_unit_amount = 0

            bought_amount = TRADE_UNIT * bought_unit_amount
            avg_jpy = used_jpy / bought_amount
            diff_jpy = now_sell_jpy - avg_jpy
            gain_jpy = now_sell_jpy * bought_amount - used_jpy
            total_gain_jpy = total_gained_jpy + gain_jpy
            fth.write(
                f'BOUGHT {please_buy_amount:.4f}=-{bought_amount-please_buy_amount:.4f}({used_jpy_before/MAX_TRADED_JPY*100:.2f}%)'
                f'+{bought_amount:.4f}({used_jpy/MAX_TRADED_JPY*100:.2f}%) ETH for {now_buy_jpy} JPY: SELL {bought_amount:.4f} ETH'
                f' for {diff_jpy:.2f}={now_sell_jpy}-{avg_jpy:.2f} JPY: NO.{trade_count+1}-{buy_count} at {datetime.now()}\n'
                f'GAIN {total_gain_jpy:+.2f}={total_gained_jpy:+.2f}{gain_jpy:+.2f} JPY in total at {datetime.now()}\n\n'
            )
        elif sample_number % TRADE_RATE == 0:
            bought_amount = TRADE_UNIT * bought_unit_amount
            sell_for_str = f'{now_sell_jpy}'
            if bought_unit_amount > 0:
                avg_jpy = used_jpy / bought_amount
                diff_jpy = now_sell_jpy - avg_jpy
                sell_for_str = f'{diff_jpy:.2f}={now_sell_jpy}-{avg_jpy:.2f}'
            gain_jpy = now_sell_jpy * bought_amount - used_jpy
            total_gain_jpy = total_gained_jpy + gain_jpy
            fth.write(
                f'BUY for {now_buy_jpy} JPY: SELL for {sell_for_str} JPY: at {datetime.now()}\n'
                f'GAIN {total_gain_jpy:+.2f}={total_gained_jpy:+.2f}{gain_jpy:+.2f} JPY in total at {datetime.now()}\n\n'
            )

        bought_amount = TRADE_UNIT * bought_unit_amount
        gain_jpy = now_sell_jpy * bought_amount - used_jpy
        total_gain_jpy = total_gained_jpy + gain_jpy
        fsh.write(
            f'BUY ETH for {now_buy_jpy} JPY: SELL ETH for {now_sell_jpy} JPY\n'
            f'GAIN {total_gain_jpy:+.2f}={total_gained_jpy:+.2f}{gain_jpy:+.2f} JPY in total at {datetime.now()}\n\n'
        )

        if sample_number % NOTIFY_RATE == 0:
            bought_amount = TRADE_UNIT * bought_unit_amount
            gain_jpy = now_sell_jpy * bought_amount - used_jpy
            total_gain_jpy = total_gained_jpy + gain_jpy
            avg_jpy_str = f'{used_jpy/bought_amount:.2f} / ' if bought_unit_amount > 0 else ''
            mock_of_str = 'Mock of ' if isinstance(prv, Mock) else ''
            send_slack(
                f'Gain {total_gain_jpy:+.2f}={total_gained_jpy:+.2f}{gain_jpy:+.2f} JPY in total by {mock_of_str}{ROBOT_NAME}'
                f' ({avg_jpy_str}{used_jpy/MAX_TRADED_JPY*100:.2f}%) at {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}\n'
                f'Buy ETH for {now_buy_jpy} JPY. Support level is {MIN_UNIT_CC_INIT_TRADE_JPY} JPY.',
                'Power by https://jhub.name/', 'good' if total_gain_jpy > 0 else 'danger'
            )

        if sample_number % BACKUP_RATE == 0:
            write_status({
                'sample_number': sample_number + 1,
                'trade_count': trade_count,
                'buy_count': buy_count,
                'sell_count': sell_count,
                'bought_unit_amount': bought_unit_amount,
                'used_jpy': used_jpy,
                'bought_average_jpy': bought_average_jpy,
                'sold_unit_amount': sold_unit_amount,
                'got_jpy': got_jpy,
                'sold_average_jpy': sold_average_jpy,
                'please_buy_unit_amount': please_buy_unit_amount,
                'please_sell_unit_amount': please_sell_unit_amount,
                'cooling_time': cooling_time,
                'total_gained_jpy': total_gained_jpy,
                'trade_unit': TRADE_UNIT
            })

        sample_number += 1
        time.sleep(SAMPLE_INTERVAL / TEST_RATIO)

In [ ]:
sell_all()